## Flask

Flask is a microframework for Python.
- http://flask.pocoo.org/

### Resources for Learning Flask

Explore Flask
- Free book funded by a kickstarter campaign:
- http://exploreflask.com/en/latest/index.html

Miguel Grinberg - Flask Workshop - PyCon 2015
- Great tutorial documenting core features here:
- Video of Workshop: https://www.youtube.com/watch?v=DIcpEg77gdE
- Code: https://github.com/miguelgrinberg/flask-pycon2015

### Today
Today we'll use our knowledge of pandas to make an insecure API for querying pandas data structures.

### Important notes on running Flask in Jupyter

- Not a good idea to run Flask in Jupyter. I'm doing it here to ease presentation of the information.
- Press the stop button above after running Flask in one of these cells before executing code in another cell.
- When Jupyter becomes unresponsive
  - `Kernel --> Restart & Clear Output`

### An example

In [ ]:
from flask import Flask

app = Flask(__name__) # Instantiate Flask

@app.route('/')       # Define a URL pattern and the function associated with it
def index():
    return '<h1>API for querying titanic data</h1>'


if __name__ == '__main__':
#     app.run(debug=True) # Debug mode does not work in Jupyter
    app.run()

### Notes
- passing `__name__` helps Flask find other files relative to this file (css, images, etc.)
- `index` name here could be anything
- `app.run()` executes the development server
  - `if __name__ == '__main__':` ensures development server not run in production

### View decorators

"Python decorators are functions that are used to transform other functions. When a decorated function is called, the decorator is called instead. The decorator can then take action, modify the arguments, halt execution or call the original function. We can use decorators to wrap views with code we’d like to run before they are executed."
- http://exploreflask.com/en/latest/views.html#view-decorators

Details on how Flask does it
- https://ains.co/blog/things-which-arent-magic-flask-part-1.html

### An example decorator

In [ ]:
# Source: https://ains.co/blog/things-which-arent-magic-flask-part-1.html

# This is our decorator
def simple_decorator(f):
    # This is the new function we're going to return
    # This function will be used in place of our original definition
    def wrapper():
        print("Entering Function")
        f()
        print("Exited Function")

    return wrapper

@simple_decorator 
def hello():
    print("Hello World")

hello()

### Hmmmm...
- How can I check that I'm understanding this right?
- Use the pythontutor

In [ ]:
# %load_ext tutormagic

In [ ]:
# %%tutor --lang python3 --tab

### More detailed example of how this works in Flask

When python interprets the file, the routes are collected by Flask providing a mapping between route and the function

In [ ]:
# Source: https://ains.co/blog/things-which-arent-magic-flask-part-1.html
class NotFlask():
    def __init__(self):
        self.routes = {}

    def route(self, route_str):
        def decorator(f):
            self.routes[route_str] = f
            return f

        return decorator

    def serve(self, path):
        view_function = self.routes.get(path)
        return view_function()

not_flask_app = NotFlask()

@not_flask_app.route("/")
def hello():
    return "Hello World!"

@not_flask_app.route("/about")
def some_name_that_doesnt_matter():
    return "Somthing about us."

not_flask_app.routes

### Hmmmm...
- How can I check that I'm understanding this right?
- Use the pythontutor

In [ ]:
# %load_ext tutormagic

In [ ]:
# %%tutor --lang python3 --tab

## Making a simple API to serve up the results of pandas queries

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import datetime
import io
import random

from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from matplotlib.dates import DateFormatter

### Dynamic routes

- Key point to remember
  - This data is coming from a user
  - You must validate the data or it could crash your server

In [ ]:
from flask import Flask

app = Flask(__name__)
df = pd.read_csv('titanic_data.csv')

@app.route("/")
def index():
    return '<h1>API for querying titanic data</h1>'

# http://127.0.0.1:5000/column/name
@app.route('/column/<string:name>')
def column_version1(name):
    return(df[name].to_json())

# http://127.0.0.1:5000/column?name=name
@app.route('/column')
def column_version2():
    name = request.args.get('name')
    return(df[name].to_json())

if __name__ == "__main__":
    app.run()

### Exercise
1. write a `/describe/name` endpoint which applies the `describe()` function to the column with name `name` and returns json to the user
- use curl to read data from the column endpoint in the terminal
- in a separate notebook or python terminal, use the requests package to read data from the column endpoint

### Adding plots

- Webservices: http://www.plot.ly

In [ ]:
from flask import Flask, request, jsonify, make_response

app = Flask(__name__)
df = pd.read_csv('titanic_data.csv')

# http://127.0.0.1:5000/hist/name
@app.route('/hist/<string:name>')
def hist(name):
    if df[name].dtype not in ('float64', 'int64'):
        return "Can't plot type: {}".format(df[name].dtype)
    
    fig = plt.figure()
    
    # Write code to generate a histogram for the column with name `name`. pass the argument `figsize=(10,10)`
    
    plt.ylabel(name)
    canvas=FigureCanvas(fig)
    png_output = io.BytesIO()
    canvas.print_png(png_output)
    response=make_response(png_output.getvalue())
    response.headers['Content-Type'] = 'image/png'
    return response

# http://127.0.0.1:5000/plot/name
@app.route('/plot/<string:name>')
def plot(name):
    if df[name].dtype not in ('float64', 'int64'):
        return "Can't plot type: {}".format(df[name].dtype)
    
    fig = plt.figure()
    
    # Write code to generate a plot for the column with name `name`. pass the argument `figsize=(10,10)`
    
    plt.ylabel(name)
    canvas=FigureCanvas(fig)
    png_output = io.BytesIO()
    canvas.print_png(png_output)
    response=make_response(png_output.getvalue())
    response.headers['Content-Type'] = 'image/png'
    return response

if __name__ == "__main__":
    app.run()

### Can we remove the duplicated code between `hist` and `plot`?
- try a decorator
- is this really a good idea though?